In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Data bahan masakan
ingredients = [
    "tomato, garlic, olive oil, spaghetti",
    "chicken, lettuce, mayonnaise",
]

In [ ]:
# Tokenisasi
tokenizer = Tokenizer()
tokenizer.fit_on_texts(ingredients)
sequences = tokenizer.texts_to_sequences(ingredients)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=1000, output_dim=64, input_length=20),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # num_classes = jumlah kategori masakan
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Input: daftar bahan (padded_sequences)
# Label: kategori resep (encoded as integers or one-hot encoded)

model.fit(padded_sequences, labels, epochs=10, validation_split=0.2)

In [ ]:
input_ingredients = ["tomato, garlic"]
input_seq = tokenizer.texts_to_sequences(input_ingredients)
input_padded = pad_sequences(input_seq, padding='post', maxlen=20)

predictions = model.predict(input_padded)
recommended_recipe = categories[np.argmax(predictions)]
print("Recommended Recipe:", recommended_recipe)


In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf

app = Flask(__name__)
model = tf.keras.models.load_model('saved_model_path')

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.json
    input_seq = tokenizer.texts_to_sequences([data['ingredients']])
    input_padded = pad_sequences(input_seq, padding='post', maxlen=20)
    predictions = model.predict(input_padded)
    recipe = categories[np.argmax(predictions)]
    return jsonify({'recommended_recipe': recipe})

if __name__ == '__main__':
    app.run()
